## 8-9. 프로젝트 - Movielens 영화 추천 실습


* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.


#### 루브릭 평가문항(* 상세기준)

1. CSR matrix가 정상적으로 만들어졌다.
* 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
* 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
* MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.	


In [79]:
# 모든 라이브러리를 한 곳에
import os
import IPython
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from implicit.als import AlternatingLeastSquares
import matplotlib.pyplot as plt

### 1. 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [80]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id','rating', 'title','timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
#ratings.rename(columns={'rating':'count'}, inplace=True)

ratings.head()

,user_id,movie_id,rating,title,timestamp
0,1,1193,5,978300760,NaN
1,1,661,3,978302109,NaN
2,1,914,3,978301968,NaN
3,1,3408,4,978300275,NaN
4,1,2355,5,978824291,NaN


In [81]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [82]:
# rating 컬럼의 이름을 viewcount로 바꿉니다.
ratings.rename(columns={'rating':'viewcount'}, inplace=True)

# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'viewcount']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,viewcount
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [83]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')

# 사용하는 컬럼만 남겨줍니다.# rating 컬럼의 이름을 viewcount로 바꿉니다.
ratings.rename(columns={'rating':'viewcount'}, inplace=True)

# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'viewcount']
ratings = ratings[using_cols]
ratings.head(10)

using_cols = ['movie_id', 'title']
movies = movies[using_cols]
movies.head()


,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### 2. 분석해 봅시다.

* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순) 

In [84]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [85]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [86]:
# * 가장 인기 있는 영화 30개(인기순)  = 많이 본 것 (영화별 시청횟수의 합)
famous_movie = ratings.groupby('movie_id')['viewcount'].sum()
rankdata = pd.merge(famous_movie.sort_values(ascending=False).head(30), movies, how='inner', on='movie_id')
rankdata

,movie_id,viewcount,title
0,2858,14449,American Beauty (1999)
1,260,13178,Star Wars: Episode IV - A New Hope (1977)
2,1196,12648,Star Wars: Episode V - The Empire Strikes Back...
3,2028,11348,Saving Private Ryan (1998)
4,1210,11303,Star Wars: Episode VI - Return of the Jedi (1983)
5,1198,11179,Raiders of the Lost Ark (1981)
6,593,11096,"Silence of the Lambs, The (1991)"
7,2571,10903,"Matrix, The (1999)"
8,2762,10703,"Sixth Sense, The (1999)"
9,589,10513,Terminator 2: Judgment Day (1991)


### 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [87]:
#아래 데이터 분석을 위해 movies파일과 ratings파일을 movie_id를 기준으로 병합
data = pd.merge(ratings, movies, on = 'movie_id', how = 'inner')
data.head()

data['title'] = data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'title','viewcount']
data = data[using_cols]
data.head(10)

,user_id,title,viewcount
0,1,one flew over the cuckoo's nest (1975),5
1,2,one flew over the cuckoo's nest (1975),5
2,12,one flew over the cuckoo's nest (1975),4
3,15,one flew over the cuckoo's nest (1975),4
4,17,one flew over the cuckoo's nest (1975),5
5,18,one flew over the cuckoo's nest (1975),4
6,19,one flew over the cuckoo's nest (1975),5
7,24,one flew over the cuckoo's nest (1975),5
8,28,one flew over the cuckoo's nest (1975),3
9,33,one flew over the cuckoo's nest (1975),5


In [88]:
my_favorite = ['back to the future (1985)' , 'fargo (1996)' ,'godfather, the (1972) ','toy story (1995)' ,'GoldenEye (1995)']
my_movielist = pd.DataFrame({"user_id":['sunny']*5, 'title':my_favorite, 'viewcount':[5.0]*5})

if not data.isin({'user_id':['sunny']})['user_id'].any():
    data = data.append(my_movielist)

data.tail(10)

,user_id,title,viewcount
836473,5851,one little indian (1973),5.0
836474,5854,slaughterhouse (1987),4.0
836475,5854,"promise, the (versprechen, das) (1994)",3.0
836476,5938,"five wives, three secretaries and me (1998)",4.0
836477,5948,identification of a woman (identificazione di ...,5.0
0,sunny,back to the future (1985),5.0
1,sunny,fargo (1996),5.0
2,sunny,"godfather, the (1972)",5.0
3,sunny,toy story (1995),5.0
4,sunny,GoldenEye (1995),5.0


### 4. CSR matrix를 직접 만들어 봅시다.

In [89]:
# index 만들기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


In [90]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['sunny'])
print(movie_to_idx['back to the future (1985)'])

6039
22


In [91]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing complete')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')

temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing complete')
    data['title'] = temp_movie_data
else:
    print('movie column indexing fail')

data

user_id column indexing complete
movie column indexing complete


,user_id,title,viewcount
0,0,0,5.0
1,1,0,5.0
2,2,0,4.0
3,3,0,4.0
4,4,0,5.0
...,...,...,...
0,6039,22,5.0
1,6039,51,5.0
2,6039,3628,5.0
3,6039,40,5.0


In [92]:
from scipy.sparse import csr_matrix


num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.viewcount, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3630 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5.als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [93]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [94]:
# 5개 영화중 하나인 X-files 영화 (movie_id: 1909)와 나 (user_id: 6041)

sunny, backtothe = user_to_idx['sunny'], movie_to_idx['back to the future (1985)']
sunny_vector, backtothe_vector = als_model.user_factors[sunny], als_model.item_factors[backtothe]


In [95]:
sunny_vector

array([-0.01021845, -0.09671833, -0.4762727 , -0.3664397 , -0.0014903 ,
        0.7682785 , -0.08175404,  0.09950263,  0.17941472,  0.38705987,
       -0.5524989 ,  0.6221994 , -0.18058112,  0.00545453,  0.6608295 ,
       -0.08679853, -0.33987287, -0.02061199, -0.26563853,  0.38588944,
       -0.15434134,  0.04101696,  0.62679446, -0.08950552, -0.18931416,
        0.3223746 , -0.44590443, -0.257934  , -0.07973496,  0.24437606,
       -0.27722475, -0.37829623,  0.0452983 , -0.08745324,  0.33561808,
       -0.18503815, -0.14567584,  0.63972485,  0.2148825 , -0.41010755,
       -0.12184649,  0.13133433,  0.30697018,  0.35329098,  0.15878317,
       -0.09628551, -0.40934637, -0.58933604,  0.91752136,  0.05694859,
       -0.00760199,  0.04337037,  0.90925217,  0.32664433, -0.7427003 ,
        0.4303226 , -0.37522593, -0.03060648, -0.00204331,  0.20456834,
        0.3000214 , -0.41891646, -0.1424576 ,  0.45421433, -0.03604973,
        0.31443912, -0.34005764, -0.27321044,  0.4085405 , -0.04

In [96]:
backtothe_vector

array([-6.44734781e-03, -1.23252498e-03, -5.95562486e-03,  2.72535831e-02,
        1.80074684e-02,  7.75029659e-02, -9.25741624e-03,  1.34122977e-02,
        3.78293060e-02,  1.26881739e-02, -9.58198775e-03,  1.20522361e-02,
        3.49382916e-03, -7.02004414e-03,  3.14934179e-02, -2.35185046e-02,
       -3.97709617e-03,  2.47942545e-02,  1.30874466e-03,  3.72773372e-02,
        7.92212202e-04,  2.62299664e-02,  2.57918853e-02,  1.50836464e-02,
        2.73412513e-03,  4.26507145e-02,  1.17758932e-02,  1.44810835e-02,
        5.62831003e-04, -1.44650219e-02, -7.84180872e-03, -7.54053844e-03,
       -1.61196701e-02, -1.12318872e-02,  5.03905341e-02, -8.45056865e-03,
        1.47647373e-02,  1.62230507e-02,  1.76040083e-02,  1.60380714e-02,
        1.11964475e-02, -2.67715678e-02,  8.73224903e-03,  5.57231688e-05,
        1.14423065e-02,  1.06940232e-02, -2.30190717e-02, -2.47652419e-02,
        4.10153121e-02,  8.27738270e-03,  1.37891760e-02,  4.68426006e-04,
        3.64309959e-02,  

In [97]:
np.dot(sunny_vector, backtothe_vector)

0.44379795

In [98]:
#그 외의 아무 영화 하나
heat = movie_to_idx[str.lower('Heat (1995)')]
heat_vector = als_model.item_factors[heat]
np.dot(sunny_vector, heat_vector)

-0.025634827

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [99]:
favorite_id = movie_to_idx[str.lower('toy story (1995)')]
similar_movie = als_model.similar_items(favorite_id, N=15)

idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'babe (1995)',
 'groundhog day (1993)',
 'GoldenEye (1995)',
 'godfather, the (1972) ',
 'lion king, the (1994)',
 'beauty and the beast (1991)',
 'pleasantville (1998)',
 "there's something about mary (1998)",
 'shakespeare in love (1998)',
 'mulan (1998)',
 'forrest gump (1994)']

In [100]:
#내가 좋아하는 영화와 비슷한 영화

def get_similar_movie(movie_title:str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie('aladdin (1992)')

['aladdin (1992)',
 'beauty and the beast (1991)',
 'lion king, the (1994)',
 'toy story (1995)',
 'little mermaid, the (1989)',
 'hunchback of notre dame, the (1996)',
 'hercules (1997)',
 'mulan (1998)',
 'anastasia (1997)',
 'tarzan (1999)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [101]:
user = user_to_idx['sunny']
movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['toy story 2 (1999)',
 'babe (1995)',
 'silence of the lambs, the (1991)',
 'e.t. the extra-terrestrial (1982)',
 'shawshank redemption, the (1994)',
 'sixth sense, the (1999)',
 'american beauty (1999)',
 'pulp fiction (1994)',
 "bug's life, a (1998)",
 'l.a. confidential (1997)',
 'groundhog day (1993)',
 'usual suspects, the (1995)',
 'goodfellas (1990)',
 'ghostbusters (1984)',
 'shakespeare in love (1998)']

In [102]:
#아이템간 유사도, 기여도
silence = movie_to_idx['silence of the lambs, the (1991)']
explain = als_model.explain(user, csr_data, itemid=silence)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('fargo (1996)', 0.2994165638976067),
 ('back to the future (1985)', 0.05845659743265043),
 ('toy story (1995)', 0.0012689926870802738),
 ('GoldenEye (1995)', 0.0010747161581807704),
 ('godfather, the (1972) ', 0.001065514085755195)]

### 총평

* data.count 로 필드명을 바꾸었다가, 예약어 때문에서 인지 csr matrix 를 만드는 과정에서 타입 오류가 지속해서 발생되었다. data.viewcount로 대체하여 해결